# Compute age-specific baselines for the STMF data  using our (Karlinsky & Kobak, 2021) model

In [1]:
%matplotlib notebook

import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
import matplotlib

from matplotlib.patches import Polygon
from sklearn.linear_model import LinearRegression
import datetime
import statsmodels.api as sm

In [2]:
df_stmf = pd.read_csv('https://www.mortality.org/Public/STMF/Outputs/stmf.csv', skiprows=2)

df_stmf

,CountryCode,Year,Week,Sex,D0_14,D15_64,D65_74,D75_84,D85p,DTotal,R0_14,R15_64,R65_74,R75_84,R85p,RTotal,Split,SplitSex,Forecast
0,AUS2,2015,1,m,5.037600,210.962400,204.0,398.0,394.0,1212.0,0.000113,0.001395,0.010716,0.041683,0.119154,0.005326,1,0,0
1,AUS2,2015,1,f,6.758007,141.241993,154.0,323.0,676.0,1301.0,0.000160,0.000929,0.007869,0.028785,0.118644,0.005641,1,0,0
2,AUS2,2015,1,b,11.795607,352.204393,358.0,721.0,1070.0,2513.0,0.000136,0.001161,0.009273,0.034714,0.118831,0.005484,1,0,0
3,AUS2,2015,2,m,5.648218,166.351782,216.0,343.0,399.0,1130.0,0.000127,0.001100,0.011347,0.035923,0.120666,0.004966,1,0,0
4,AUS2,2015,2,f,6.983274,149.016726,147.0,290.0,646.0,1239.0,0.000166,0.000980,0.007511,0.025844,0.113378,0.005372,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108691,USA,2021,46,f,205.000000,5852.000000,5478.0,7494.0,10552.0,29581.0,0.000367,0.002867,0.015926,0.040556,0.125388,0.009205,0,0,1
108692,USA,2021,46,b,451.000000,16018.000000,13015.0,15641.0,17693.0,62818.0,0.000394,0.003925,0.020190,0.047388,0.133529,0.009921,0,0,1
108693,USA,2021,47,m,303.000000,9985.000000,7571.0,8151.0,7207.0,33217.0,0.000518,0.004896,0.025182,0.056105,0.149063,0.010652,0,0,1
108694,USA,2021,47,f,202.000000,5907.000000,5480.0,7414.0,10625.0,29628.0,0.000361,0.002893,0.015931,0.040123,0.126256,0.009219,0,0,1


In [3]:
def predict(X):    
    # Fit regression model on pre-2020 data from 2015 on 
    ind = (X[:,0] >= 2015) & (X[:,0] < 2020) & (X[:,1]<53)
    m = int(np.max(X[ind,1]))
    onehot = np.zeros((np.sum(ind), m))
    for i,k in enumerate(X[ind,1]):
        onehot[i,int(k)-1] = 1
    predictors = np.concatenate((X[ind,:1], onehot), axis=1)
    reg = LinearRegression(fit_intercept=False).fit(predictors, X[ind,2])
            
    # Compute 2020 baseline
    ind2 = X[:,0] == 2020
    predictors2020 = np.concatenate((np.ones((m,1))*2020, np.eye(m)), axis=1)
    baseline = reg.predict(predictors2020)
    
    # Week 53 usually does not have enough data, so we'll use 
    # the same baseline value as for week 52
    baseline = np.concatenate((baseline, [baseline[-1]]))
    
    # Compute 2021 baseline
    predictors2021 = np.concatenate((np.ones((m,1))*2021, np.eye(m)), axis=1)
    baseline2021 = reg.predict(predictors2021)
    
    # Compute 2022 baseline
    predictors2022 = np.concatenate((np.ones((m,1))*2022, np.eye(m)), axis=1)
    baseline2022 = reg.predict(predictors2022)
    
    return baseline, baseline2021, baseline2022

In [4]:
countries = np.unique(df_stmf['CountryCode'])
agebands = ['D0_14', 'D15_64', 'D65_74', 'D75_84', 'D85p', 'DTotal']
sexes = ['m', 'f', 'b']

with open('baselines-stmf.csv', 'w') as f:
    for country in countries:
        print('.', end='')
        for ageband in agebands:
            for sex in sexes:
                X = df_stmf[(df_stmf['CountryCode']==country) & (df_stmf['Sex']==sex)]
                X = X[['Year','Week',ageband]].values
                baselines  = predict(X)

                for year in range(2020,2023): 
                    for i,b in enumerate(baselines[year-2020]):
                        f.write(f'{country}, {year}, {ageband}, {sex}, {i+1}, {b:.1f}\n')
print('')

......................................
